## Import Required Libraries

We'll import all necessary Python libraries for financial data analysis, portfolio optimization, and visualization.

In [1]:
import warnings
import os
#from datetime import datetime, timedelta
import json
import pandas as pd
#import numpy as np
#import random
#import sys
from IPython.display import display, Markdown
#sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..', 'AssetPricingPortfolio')))

from Backtester.helpers import read_datasets_data,recursive_serialize

from Backtester.BacktestFramework import BacktraderPortfolioBacktest

from Backtester.Strategy_Core import EqualWeightStrategy #, InverseVolatilityStrategy,QuintileStrategy,GMVPStrategy,MarkowitzStrategy,VanillaRiskParityStrategy,MaximumSharpeRatioStrategy,MostDiversifiedStrategy,MaximumDecorrelationStrategy
from Backtester.Strategy_AAA import AdaptiveAssetAllocationStrategy
from Backtester.Strategy_dual_momentum import GlobalEquitiesMomentumStrategy

warnings.filterwarnings('ignore')

# Define settings, load universe data and prepare resampled datasets for backtest

In [2]:
# define backtest settings
universe_name = "selection3" 
test_name = "AAA"

In [3]:
#load the test data

universe_folder_path = os.path.join("data",universe_name)
datasets_folder_path = os.path.join("data",universe_name,test_name,"datasets")
test_folder_path = os.path.join("data",universe_name,test_name)

with open(os.path.join(universe_folder_path, 'universe_settings.json'), 'r') as f:
    universe_settings = json.load(f)

with open(os.path.join(test_folder_path, 'datasets_settings.json'), 'r') as f:
    datasets_settings = json.load(f)

with open(os.path.join(test_folder_path, 'test_settings.json'), 'r') as f:
    test_settings = json.load(f)

universe_info_df = pd.read_csv(os.path.join(universe_folder_path, "universe_info.csv"), header=[0], index_col=0, parse_dates=True)
universe_data = pd.read_csv(os.path.join(universe_folder_path, "universe_data.csv"), header=[0,1], index_col=0, parse_dates=True)

datasets_info, my_dataset_list = read_datasets_data(test_folder_path)

backtest_settings = {
    'lookback':datasets_settings['lookback_periods'],
}

print(backtest_settings)  


Loaded 100 datasets from data\selection3\AAA
{'lookback': 63}


## Run Backtest

We'll implement a comprehensive backtesting framework that includes:
- Portfolio rebalancing at specified intervals
- Benchmark portfolios (1/N and index)
- Performance metrics calculation
- Transaction costs simulation

### Available strategies

"Quintile":
"Markowitz":
"GMVP":
"IVP": (InverseVolatilityStrategy),
"Vanilla_RPP": (VanillaRiskParityStrategy),
"MSRP": (MaximumSharpeRatioStrategy),
"MDP": (MostDiversifiedStrategy),
"MCDP": (MaximumDecorrelationStrategy)



### common parameters

- 'strategies',
- 'datasets',
- 'benchmark'=['1/N'], 
- 'rebalance_every'=21*1,
- 'lookback'=252//3,
- 'warmup'=None,
- 'initial_cash'=100000,
- 'commission'=0.001

### Per Strategy parameters:

- `lookback`: Historical data window for portfolio calculations
- `rebalance_every`: Frequency of portfolio rebalancing
- 'warmup'=None,             # Warmup period (defaults to lookback if None)
- 'portfolio_func'=None,     # Portfolio optimization function
- 'dataset_assets'=None,     # List of asset names
- 'buy_slippage_buffer'=0.01 # Extra cushion to protect against price surprises on buys

This ensures that when portfolio optimization functions (like Markowitz, GMVP) are called, they have the required historical data for meaningful calculations.

In [4]:
backtest_settings['rebalance_every'] = 5  # in days
backtest_settings['initial_cash'] = 100000
backtest_settings['commission'] = 0.001  # 0.1% commission
backtest_settings['short_interest'] = 0.03  # 3% annual short interest
backtest_settings['interest_long'] = False  # Only charge shorts
backtest_settings['checksubmit'] = False

display(Markdown(f"### Backtest Settings\n```json\n{json.dumps(backtest_settings, indent=4)}\n```"))

try:
    with open(os.path.join(test_folder_path, 'strategies.json'), 'r') as f:
        existing_strategies = json.load(f)
    #display(Markdown(f"### Existing Strategies\n```json\n{json.dumps(existing_strategies, indent=4)}\n```"))
    display(existing_strategies)

except FileNotFoundError:
    existing_strategies = []

### Backtest Settings
```json
{
    "lookback": 63,
    "rebalance_every": 5,
    "initial_cash": 100000,
    "commission": 0.001,
    "short_interest": 0.03,
    "interest_long": false,
    "checksubmit": false
}
```

[{'EqualWeight': ['MetaStrategy', {'buy_slippage_buffer': 0.01}]},
 {'AAA1': ['MetaStrategy',
   {'buy_slippage_buffer': 0.04,
    'portfolio_func_kwargs': {'momentum_top_n': 5,
     'momentum_lookback': 21,
     'covariance_lookback': 10}}]},
 {'AAA2': ['MetaStrategy',
   {'buy_slippage_buffer': 0.04,
    'portfolio_func_kwargs': {'momentum_top_n': 7,
     'momentum_lookback': 42,
     'covariance_lookback': 21}}]},
 {'AAA3': ['MetaStrategy',
   {'buy_slippage_buffer': 0.02,
    'use_stops': True,
    'trailing_stop': True,
    'trail_percent': 0.05,
    'verbose': False,
    'portfolio_func_kwargs': {'momentum_top_n': 10,
     'momentum_lookback': 42,
     'covariance_lookback': 21}}]},
 {'AAA4': ['MetaStrategy',
   {'buy_slippage_buffer': 0.02,
    'use_stops': True,
    'trailing_stop': True,
    'trail_percent': 0.05,
    'verbose': True,
    'portfolio_func_kwargs': {'momentum_top_n': 5,
     'momentum_lookback': 21,
     'covariance_lookback': 10}}]},
 {'AAA5': ['MetaStrategy',


In [6]:
bt_strategies = {
    #"EqualWeight": (EqualWeightStrategy, {
    #    'buy_slippage_buffer': 0.01,
    #}),
    #'GEM4': (GlobalEquitiesMomentumStrategy, {
    #    'buy_slippage_buffer': 0.02,
    #    'use_stops': True,
    #    'trailing_stop': True,
    #    'trail_percent': 0.05,  # 5% trailing stop
    #    'verbose': False,
    #    'portfolio_func_kwargs': {
    #        'allow_short': False,
    #        'momentum_periods': [5, 10, 21, 21*3-1],  # Days, not months!
    #        'min_positive_periods': 2,
    #        'treasury_threshold': 0.0,
    #        'maximum_positions': 5,
    #        'risk_free_asset': 'cash',  # 'cash' (default) or symbol like 'BIL', 'SHY'
    #    }
    #}),
    #"AAA1": (AdaptiveAssetAllocationStrategy, {
    #    'buy_slippage_buffer': 0.04,
    #    'portfolio_func_kwargs': {
    #        'momentum_top_n': 5, 
    #        'momentum_lookback': 21,
    #        'covariance_lookback': 10,
    #    }
    #}),
    #"AAA2": (AdaptiveAssetAllocationStrategy, {
    #    'buy_slippage_buffer': 0.04,
    #    'portfolio_func_kwargs': {
    #        'momentum_top_n': 7, 
    #        'momentum_lookback': 21*2,
    #        'covariance_lookback': 21,
    #    }
    #}),
    #"AAA3": (AdaptiveAssetAllocationStrategy, {
    #    'buy_slippage_buffer': 0.02,
    #    'use_stops': True,
    #    'trailing_stop': True,
    #    'trail_percent': 0.05,  # 5% trailing stop
    #    'verbose': True,
    #    'portfolio_func_kwargs': {
    #        'momentum_top_n': 10, 
    #        'momentum_lookback': 21*2,
    #        'covariance_lookback': 21,
    #    }
    #}),
    #"AAA4": (AdaptiveAssetAllocationStrategy, {
    #    'buy_slippage_buffer': 0.02,
    #    'use_stops': True,
    #    'trailing_stop': True,
    #    'trail_percent': 0.05,  # 5% trailing stop
    #    'verbose': True,
    #    'portfolio_func_kwargs': {
    #        'momentum_top_n': 5, 
    #        'momentum_lookback': 21,
    #        'covariance_lookback': 10,
    #    }
    #}),
    "AAA9": (AdaptiveAssetAllocationStrategy, {
        'buy_slippage_buffer': 0.02,
        'use_stops': True,
        'trailing_stop': True,
        'trail_percent': 0.03,  # 3% trailing stop
        'verbose': False,
        'portfolio_func_kwargs': {
            'momentum_top_n': 10, 
            'momentum_lookback': 21,
            'covariance_lookback': 21*3-1,
        }
    }),
    "AAA10": (AdaptiveAssetAllocationStrategy, {
        'buy_slippage_buffer': 0.02,
        'use_stops': False,
        'trailing_stop': False,
        'trail_percent': 0.05,  # 10% trailing stop
        'verbose': False,
        'portfolio_func_kwargs': {
            'momentum_top_n': 10, 
            'momentum_lookback': 21,
            'covariance_lookback': 21*3-1,
        }
    }),
}

existing_strategies.append(recursive_serialize(bt_strategies))
with open(os.path.join(test_folder_path, 'strategies.json'), 'w') as f:
    json.dump(existing_strategies, f, indent=4)

In [7]:
bt_backtest = BacktraderPortfolioBacktest(
    strategies=bt_strategies, 
    datasets=my_dataset_list,
    rebalance_every=backtest_settings['rebalance_every'],  # in days
    lookback=int(backtest_settings['lookback']),  # in days
    initial_cash=backtest_settings['initial_cash'],
    commission=backtest_settings['commission'],
    short_interest=backtest_settings['short_interest'],
    interest_long=backtest_settings['interest_long'],
    #checksubmit=False,
)

# Run the backtest
backtest_results = bt_backtest.run_backtest()

print("\nBacktrader backtesting with warmup period completed successfully!")
print(f"Lookback period: {bt_backtest.lookback} days")
print(f"Rebalancing frequency: {bt_backtest.rebalance_every} days")

# save results

serializable_results = recursive_serialize(backtest_results)

results_folder_path = os.path.join(test_folder_path, 'results')
os.makedirs(results_folder_path, exist_ok=True)

for strategy, datasets in serializable_results.items():
    strategy_results = {}
    strategy_results['Strategy'] = strategy
    strategy_results['Datasets'] = datasets
    with open(os.path.join(results_folder_path, f'{strategy}_results.json'), 'w') as f:
        json.dump(strategy_results, f, indent=2)


Backtrader backtesting with warmup period completed successfully!
Lookback period: 63 days
Rebalancing frequency: 5 days


In [7]:
"""
    #"Quintile": (QuintileStrategy, {
    #    'buy_slippage_buffer': 0.07,
        #'checksubmit': False, 
    #}),
    "MVP": (MarkowitzStrategy, {
        'buy_slippage_buffer': 0.07,
        'portfolio_func_kwargs': {'allow_short': False}
        #'checksubmit': False,
    }),

    "GMVP": (GMVPStrategy, {
        'buy_slippage_buffer': 0.05,
        'portfolio_func_kwargs': {'allow_short': False}
    }),
    "IVP": (InverseVolatilityStrategy, {
        'buy_slippage_buffer': 0.04,
    }),
    "RPP": (VanillaRiskParityStrategy, {
        'buy_slippage_buffer': 0.04,
    }),
    "MSRP": (MaximumSharpeRatioStrategy, {
        'buy_slippage_buffer': 0.05,
        'portfolio_func_kwargs': {'allow_short': False}
    }),
    "MDP": (MostDiversifiedStrategy, {
        'buy_slippage_buffer': 0.04,
        'portfolio_func_kwargs': {'allow_short': False}
    }),
    "MDC": (MaximumDecorrelationStrategy, {
        'buy_slippage_buffer': 0.04,
        'portfolio_func_kwargs': {'allow_short': False}
    }),
    "EqualWeight": (EqualWeightStrategy, {
        'buy_slippage_buffer': 0.03,
    }),
"""

'\n    #"Quintile": (QuintileStrategy, {\n    #    \'buy_slippage_buffer\': 0.07,\n        #\'checksubmit\': False, \n    #}),\n    "MVP": (MarkowitzStrategy, {\n        \'buy_slippage_buffer\': 0.07,\n        \'portfolio_func_kwargs\': {\'allow_short\': False}\n        #\'checksubmit\': False,\n    }),\n\n    "GMVP": (GMVPStrategy, {\n        \'buy_slippage_buffer\': 0.05,\n        \'portfolio_func_kwargs\': {\'allow_short\': False}\n    }),\n    "IVP": (InverseVolatilityStrategy, {\n        \'buy_slippage_buffer\': 0.04,\n    }),\n    "RPP": (VanillaRiskParityStrategy, {\n        \'buy_slippage_buffer\': 0.04,\n    }),\n    "MSRP": (MaximumSharpeRatioStrategy, {\n        \'buy_slippage_buffer\': 0.05,\n        \'portfolio_func_kwargs\': {\'allow_short\': False}\n    }),\n    "MDP": (MostDiversifiedStrategy, {\n        \'buy_slippage_buffer\': 0.04,\n        \'portfolio_func_kwargs\': {\'allow_short\': False}\n    }),\n    "MDC": (MaximumDecorrelationStrategy, {\n        \'buy_slippag